In [168]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
from seaborn import set_style
set_style("whitegrid")

In [146]:
## For the model
import xgboost 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [169]:
news_train = pd.read_pickle("C:/Users/HJMit/Desktop/train_PCA_cols.pkl")
#news_val = pd.read_pickle("C:/Users/HJMit/Desktop/train_PCA_cols.pkl")

In [149]:
news_train.head(2)

,index,Unnamed: 0,link_col,headline_col,category_col,short_description,authors_col,date_col,headline_cleaned,short_description_cleaned,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
0,0,0,https://www.huffingtonpost.com/entry/blgt-usas...,BLgT USA's Latest Pit Stop On Its LGBTQ Equali...,QUEER VOICES,The latest updates from the 50-state food equa...,James Michael Nichols,2015-07-13,blgt usa late pit stop lgbtq equality tour south,late update 50 state food equality tour,...,0.025050,-0.018328,-0.003255,0.006662,-0.001935,-0.002674,-0.002762,-0.014888,0.010297,-0.007270
1,1,1,https://www.huffingtonpost.com/entry/let-go_us...,Legacy: Learning to Let Go,WELLNESS,September brings its newness. Energy is high a...,"Rachael Freed, Contributor\r\ncreator of life ...",2013-09-06,legacy learn let go,september bring newness energy high teacher ch...,...,0.012558,-0.005345,0.010755,0.003383,-0.007317,-0.022330,0.006550,0.006965,-0.005208,-0.004315


In [150]:
# news_train.drop('Unnamed: 0', axis=1, inplace=True)
# news_train.drop('index', axis=1, inplace=True)

In [ ]:
#news_val.drop('Unnamed: 0', axis=1, inplace=True)
# news_val.drop('index', axis=1, inplace=True)

In [170]:
## clean up indices and do one-hot encoding
dat_list = [news_train, news_val]

for news in dat_list:
    
    ## get rid of excess indices
    news.drop('Unnamed: 0', axis=1, inplace=True)
    news.drop('index', axis=1, inplace=True)
    
    ## One-hot encoding for category cols 
    news['politics'] = 0
    news.loc[news.category_col=='POLITICS', 'politics']=1

    news['wellness'] = 0
    news.loc[news.category_col=='WELLNESS', 'wellness']=1

    news['entertainment'] = 0
    news.loc[news.category_col=='ENTERTAINMENT', 'entertainment']=1

    news['parenting'] = 0
    news.loc[news.category_col=='PARENTING', 'parenting']=1

    news['style'] = 0
    news.loc[news.category_col=='STYLE & BEAUTY', 'style']=1

    news['travel'] = 0
    news.loc[news.category_col=='TRAVEL', 'travel']=1

    news['food'] = 0
    news.loc[news.category_col=='FOOD & DRINK', 'food']=1

    news['queer_voices'] = 0
    news.loc[news.category_col=='QUEER VOICES', 'queer_voices']=1

    news['business'] = 0
    news.loc[news.category_col=='BUSINESS', 'business']=1

    news['sports'] = 0
    news.loc[news.category_col=='SPORTS', 'sports']=1

    news['black_voices'] = 0
    news.loc[news.category_col=='BLACK VOICES', 'black_voices']=1

In [163]:
### Columns which only have PCA Cols

pca_cols = news_train.columns[9:-11]
X_train  = news_train

pca_cols_v = news_val.columns[9:-11]
X_val = news_val


print(X_train.head(2),
      X_val.head(2))

,link_col,headline_col,category_col,short_description,authors_col,date_col,headline_cleaned,short_description_cleaned,cleaned_words,0,...,wellness,entertainment,parenting,style,travel,food,queer_voices,business,sports,black_voices
0,https://www.huffingtonpost.com/entry/blgt-usas...,BLgT USA's Latest Pit Stop On Its LGBTQ Equali...,QUEER VOICES,The latest updates from the 50-state food equa...,James Michael Nichols,2015-07-13,blgt usa late pit stop lgbtq equality tour south,late update 50 state food equality tour,blgt usa late pit stop lgbtq equality tour sou...,0.003390,...,0,0,0,0,0,0,1,0,0,0
1,https://www.huffingtonpost.com/entry/let-go_us...,Legacy: Learning to Let Go,WELLNESS,September brings its newness. Energy is high a...,"Rachael Freed, Contributor\r\ncreator of life ...",2013-09-06,legacy learn let go,september bring newness energy high teacher ch...,legacy learn let go september bring newness en...,-0.035661,...,1,0,0,0,0,0,0,0,0,0
2,https://www.huffingtonpost.com/entry/3-weirdly...,3 Weirdly Accurate Reasons to Stop Saving for ...,BUSINESS,If you're making contributions to retirement p...,"Garrett B. Gunderson, ContributorCo-founder an...",2014-12-25,3 weirdly accurate reason stop saving retireme...,'re make contribution retirement plan n't thre...,3 weirdly accurate reason stop saving retireme...,-0.018476,...,0,0,0,0,0,0,0,1,0,0
3,https://www.huffingtonpost.com/entry/mel-b-thr...,"Mel B Throws Water At Simon Cowell, Exits Afte...",ENTERTAINMENT,"""I told him he was an arsehole.""",Ron Dicker,2017-08-23,mel b throw water simon cowell exit marriage j...,`` tell arsehole '',mel b throw water simon cowell exit marriage j...,-0.002065,...,0,1,0,0,0,0,0,0,0,0
4,https://www.huffingtonpost.com/entry/yoga-immu...,Yoga Associated With Gene Expression In Immune...,WELLNESS,And although it's long been known that yoga ca...,Carolyn Gregoire,2013-04-25,yoga associate gene expression immune cell stu...,although long know yoga reduce stress ucla stu...,yoga associate gene expression immune cell stu...,-0.043957,...,1,0,0,0,0,0,0,0,0,0


In [165]:
### columns which only have category names
cat_cols = news.columns[-11:]
y_train = news_train[cat_cols]

cat_cols_v = news_val.columns[-11:] 
y_val = news_val[cat_cols_v ]

print(y_train.head(2),
     y_val.head(2))


,politics,wellness,entertainment,parenting,style,travel,food,queer_voices,business,sports,black_voices
0,0,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
104646,0,0,1,0,0,0,0,0,0,0,0
104647,0,0,0,0,0,0,1,0,0,0,0
104648,0,0,0,0,0,0,1,0,0,0,0
104649,0,0,0,0,0,0,1,0,0,0,0


In [167]:
## Train_test_split 
from sklearn.model_selection import train_test_split

X_ttrain, X_ho, y_ttrain, y_ho = train_test_split(X_train[pca_cols],y_train, 
                                         random_state = 546,
                                         shuffle = True,
                                         test_size = .1,
                                         stratify = X_train['category_col'])


# Model: Boosted Decision Trees 

### Tuning the hyperparameters - round 1

In [ ]:
max_depths = [10, 50, 100]
child_weights = [0.5, 1, 2]
max_leaves = [0, 100, 500]

cl_accs = np.zeros((len(max_depths), len(child_weights), len(max_leaves)))

best_acc = 0
best_params =  None

j = 0
for max_depth in max_depths:
    k = 0
    for weight in child_weights:
        i = 0
        for leaf in max_leaves:
            print(j,k,i)
            xgb_cl = xgboost.XGBClassifier(learning_rate=.05,
                                           max_depth=max_depth,
                                           min_child_weight=weight,
                                           max_leaves = leaf,
                                           n_estimators = 200)

            xgb_cl.fit(X_ttrain, y_ttrain,
               early_stopping_rounds = 25,  
               eval_set=[(X_ho, y_ho)])

            pred = xgb_cl.predict(X_ho)

            cl_accs[j,k,i] = accuracy_score(y_ho,  pred)

            if cl_accs[j,k,i] > best_acc:
                best_acc = cl_accs[j,k,i]
                best_params = [max_depth, weight, leaf]
            
            i = i + 1
        k = k + 1
    j = j + 1 

0 0 0


C:\Users\HJMit\anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.65571
[1]	validation_0-logloss:0.62255
[2]	validation_0-logloss:0.59257
[3]	validation_0-logloss:0.56574
[4]	validation_0-logloss:0.54087
[5]	validation_0-logloss:0.51833
[6]	validation_0-logloss:0.49779
[7]	validation_0-logloss:0.47903
[8]	validation_0-logloss:0.46079
[9]	validation_0-logloss:0.44479
[10]	validation_0-logloss:0.42998
[11]	validation_0-logloss:0.41605
[12]	validation_0-logloss:0.40326
[13]	validation_0-logloss:0.39147
[14]	validation_0-logloss:0.38047
[15]	validation_0-logloss:0.36991
[16]	validation_0-logloss:0.36054
[17]	validation_0-logloss:0.35104
[18]	validation_0-logloss:0.34253
[19]	validation_0-logloss:0.33431
[20]	validation_0-logloss:0.32711
[21]	validation_0-logloss:0.31911
[22]	validation_0-logloss:0.31252
[23]	validation_0-logloss:0.30656
[24]	validation_0-logloss:0.30078
[25]	validation_0-logloss:0.29513
[26]	validation_0-logloss:0.28964
[27]	validation_0-logloss:0.28453
[28]	validation_0-logloss:0.27988
[29]	validation_0-loglos

C:\Users\HJMit\anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.65571
[1]	validation_0-logloss:0.62255
[2]	validation_0-logloss:0.59257
[3]	validation_0-logloss:0.56574
[4]	validation_0-logloss:0.54087
[5]	validation_0-logloss:0.51833
[6]	validation_0-logloss:0.49779
[7]	validation_0-logloss:0.47903
[8]	validation_0-logloss:0.46079
[9]	validation_0-logloss:0.44479
[10]	validation_0-logloss:0.42998
[11]	validation_0-logloss:0.41605
[12]	validation_0-logloss:0.40326
[13]	validation_0-logloss:0.39147
[14]	validation_0-logloss:0.38047
[15]	validation_0-logloss:0.36991
[16]	validation_0-logloss:0.36054
[17]	validation_0-logloss:0.35104
[18]	validation_0-logloss:0.34253
[19]	validation_0-logloss:0.33431
[20]	validation_0-logloss:0.32711
[21]	validation_0-logloss:0.31911
[22]	validation_0-logloss:0.31252
[23]	validation_0-logloss:0.30656
[24]	validation_0-logloss:0.30078
[25]	validation_0-logloss:0.29513
[26]	validation_0-logloss:0.28964
[27]	validation_0-logloss:0.28453
[28]	validation_0-logloss:0.27988
[29]	validation_0-loglos

...

[0]	validation_0-logloss:0.65648
[1]	validation_0-logloss:0.62402
[2]	validation_0-logloss:0.59471
[3]	validation_0-logloss:0.56771
[4]	validation_0-logloss:0.54340
[5]	validation_0-logloss:0.52163
[6]	validation_0-logloss:0.50113
[7]	validation_0-logloss:0.48282
[8]	validation_0-logloss:0.46451
[9]	validation_0-logloss:0.44823
[10]	validation_0-logloss:0.43316
[11]	validation_0-logloss:0.41885
[12]	validation_0-logloss:0.40643
[13]	validation_0-logloss:0.39402
[14]	validation_0-logloss:0.38228
[15]	validation_0-logloss:0.37220
[16]	validation_0-logloss:0.36255
[17]	validation_0-logloss:0.35312
[18]	validation_0-logloss:0.34470
[19]	validation_0-logloss:0.33691
[20]	validation_0-logloss:0.32904
[21]	validation_0-logloss:0.32172
[22]	validation_0-logloss:0.31504
[23]	validation_0-logloss:0.30826
[24]	validation_0-logloss:0.30253
[25]	validation_0-logloss:0.29718
[26]	validation_0-logloss:0.29179
[27]	validation_0-logloss:0.28654
[28]	validation_0-logloss:0.28190
[29]	validation_0-loglos

C:\Users\HJMit\anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.65648
[1]	validation_0-logloss:0.62402
[2]	validation_0-logloss:0.59471
[3]	validation_0-logloss:0.56771
[4]	validation_0-logloss:0.54340
[5]	validation_0-logloss:0.52163
[6]	validation_0-logloss:0.50113
[7]	validation_0-logloss:0.48282
[8]	validation_0-logloss:0.46451
[9]	validation_0-logloss:0.44823
[10]	validation_0-logloss:0.43316
[11]	validation_0-logloss:0.41885
[12]	validation_0-logloss:0.40643
[13]	validation_0-logloss:0.39402
[14]	validation_0-logloss:0.38228
[15]	validation_0-logloss:0.37220
[16]	validation_0-logloss:0.36255
[17]	validation_0-logloss:0.35312
[18]	validation_0-logloss:0.34470
[19]	validation_0-logloss:0.33691
[20]	validation_0-logloss:0.32904
[21]	validation_0-logloss:0.32172
[22]	validation_0-logloss:0.31504
[23]	validation_0-logloss:0.30826
[24]	validation_0-logloss:0.30253
[25]	validation_0-logloss:0.29718
[26]	validation_0-logloss:0.29179
[27]	validation_0-logloss:0.28654
[28]	validation_0-logloss:0.28190
[29]	validation_0-loglos

C:\Users\HJMit\anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.65648
[1]	validation_0-logloss:0.62402
[2]	validation_0-logloss:0.59471
[3]	validation_0-logloss:0.56771
[4]	validation_0-logloss:0.54340
[5]	validation_0-logloss:0.52163
[6]	validation_0-logloss:0.50113
[7]	validation_0-logloss:0.48282
[8]	validation_0-logloss:0.46451
[9]	validation_0-logloss:0.44823
[10]	validation_0-logloss:0.43316
[11]	validation_0-logloss:0.41885
[12]	validation_0-logloss:0.40643
[13]	validation_0-logloss:0.39402
[14]	validation_0-logloss:0.38228
[15]	validation_0-logloss:0.37220
[16]	validation_0-logloss:0.36255
[17]	validation_0-logloss:0.35312
[18]	validation_0-logloss:0.34470
[19]	validation_0-logloss:0.33691
[20]	validation_0-logloss:0.32904
[21]	validation_0-logloss:0.32172
[22]	validation_0-logloss:0.31504
[23]	validation_0-logloss:0.30826
[24]	validation_0-logloss:0.30253
[25]	validation_0-logloss:0.29718
[26]	validation_0-logloss:0.29179
[27]	validation_0-logloss:0.28654
[28]	validation_0-logloss:0.28190
[29]	validation_0-loglos

In [135]:
# Get best parameters, plus the accuracy score for that combination
print(f"""Best accuracy is {best_acc}
Best parameters are {best_params}""")

Best accuracy is 0.36
Best parameters are [2, 1]


### Make the 1st model

In [ ]:
#Make the model with these parameters
xgb_cl = xgboost.XGBClassifier(learning_rate=.05,
                               max_depth=best_params[0],
                               min_child_weight=best_params[1],
                               max_leaves = best_params[2],
                               n_estimators = 200)

xgb_cl.fit(X_train, y_train,
   early_stopping_rounds = 25,  
   eval_set=[(X_train, y_train), (X_val, y_val)])

In [ ]:
# make predictions for val data
y_pred = xgb_cl.predict(X_val)
predictions = [value for value in y_pred]

# evaluate predictions 
accuracy = accuracy_score(y_val, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

f1 = f1_score(y_ho, predictions, average='weighted')
print("f1 score: %.2f%%" % (f1 * 100.0))

### Plot the logloss scores

In [ ]:
# Plot the logloss 

results = xgb_cl.evals_result()

plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["logloss"], label="Training loss")
plt.plot(results["validation_1"]["logloss"], label="Validation loss") 
plt.xlabel("Number of trees", fontsize = 16)
plt.ylabel("Loss", fontsize = 16)
plt.legend()
plt.title("Model 1", fontsize = 20)

plt.show()

## Rinse and repeat